<a href="https://colab.research.google.com/github/sallywang147/llm_invariants/blob/master/%5BEvaluation%5D_llama_for_smart_contracts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [16]:
!pip install --quiet accelerate
!pip install -i --quiet https://test.pypi.org/simple/ bitsandbytes
!pip install --quiet sentencepiece
!pip install --quiet git+https://github.com/huggingface/transformers
!pip install --quiet rich[jupyter]
!pip install --quiet deepspeed
!pip install --quiet openai
!pip install --quiet gradio
!pip install --quiet peft
!pip install --quiet --upgrade datasets
!pip install --quiet simplejson
!pip install --quiet accelerate
!pip install --quiet bitsandbytes

Looking in indexes: --quiet, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 76.0 MB/s eta 0:00:00
  ERROR: Cannot unpack file /tmp/pip-unpack-x9z7r1p9/simple.html (downloaded from /tmp/pip-req-build-8t7summn, content-type: text/html); cannot detect archive format
ERROR: Cannot determine archive format of /tmp/pip-req-build-8t7summn
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


##finetuned stanford-alpaca Llama v. 8bit_Quantized Llama##

vanilla llama is slightly more capable than quantized llama, but not by much. For example, vanilla llama tends to repeat the same information less. 

##Inference with stanford_alpaca trained llama##

Let's make some inferences without targeted instruction finetuning yet

In [ ]:
import os
import numpy as np
import accelerate
import bitsandbytes
import transformers
import pandas as pd
import plotly.express as px
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import AutoTokenizer, LlamaTokenizer, LlamaForCausalLM

alpaca_model = LlamaForCausalLM.from_pretrained(
            "sallywww/trained_llama_stanford_format" )

alpaca_tokenizer = LlamaTokenizer.from_pretrained(
            "sallywww/trained_llama_stanford_format"
        )
#model = LlamaForCausalLM.from_pretrained('sallywww/Llama-7B')
#tokenizer = AutoTokenizer.from_pretrained('sallywww/Llama-7B')

#model.push_to_hub('sallywww/Llama-7B')
#tokenizer.push_to_hub('sallywww/Llama-7B')


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-a100-s-2v594eetpeyxr --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=true'), PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
prompt = "a loop invariant of smart contracts is " 
inputs = alpaca_tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = alpaca_model.generate(inputs.input_ids, max_length=100)
alpaca_tokenizer.batch_decode(generate_ids, 
                       skip_special_tokens=True, 
                       temperature=0.8, 
                       top_p=0.95, 
                       top_k=50, 
                       repeat_penalty=0.2,
                       clean_up_tokenization_spaces=False)[0]


' a loop invariant of smart contracts is  assert(x == y);'

In [ ]:
contract = 'Replica.sol'
test_file = open(f'/content/drive/MyDrive/experiments/baseline benchmark/{contract}', "r")
test_data = test_file.read()

In [ ]:
import argparse
import random
import torch
#import gradio as gr
import torch.nn as nn
#import bitsandbytes
from transformers import LlamaForCausalLM, LlamaTokenizer

def infer_invariants(model, tokenizer, test_data, \
                     temperature, top_p, top_k,  max_new_tokens):
  model.cuda()
  model.eval()
  inputs = tokenizer(test_data, return_tensors="pt")
  with torch.no_grad():
      outputs = model.generate(input_ids=inputs["input_ids"].cuda(), \
       temperature=temperature, 
       top_p=top_p, 
       top_k=top_k, 
      max_new_tokens=max_new_tokens)
      invariants = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
  return invariants

In [ ]:
output = infer_invariants(model=alpaca_model, 
                          tokenizer=alpaca_tokenizer, 
                          test_data=test_data,
                          temperature=None, 
                          top_p=None, 
                          top_k=None, 
                          max_new_tokens=670)
print(output.strip())
with open(f"/content/drive/MyDrive/experiments/evaluation/noPEFT_finetuned_llama/{contract}", "w", encoding="utf-8") as file:  
    file.write(output.strip()+"\n")
file.close()

Token indices sequence length is longer than the specified maximum sequence length for this model (3200 > 512). Running this sequence through the model will result in indexing errors


1 // SPDX-License-Identifier: MIT OR Apache-2.0
2 //test cross bridge
3 pragma solidity >=0.6.11;


4 contract Replica {
 
5     /**
6      * @notice Emitted when message is processed
7      * @param messageHash Hash of message that failed to process
8      * @param success TRUE if the call was executed successfully, FALSE if the call reverted
9      * @param returnData the return data from the external call
10      */
11     event Process(
12         bytes32 indexed messageHash,
13         bool indexed success,
14         bytes indexed returnData
15     );

   

16     function initialize(
17         uint32 _remoteDomain,
18         address _updater,
19         bytes32 _committedRoot,
20         uint256 _optimisticSeconds
21     ) public initializer {
22         __NomadBase_initialize(_updater);
23         // set storage variables
24         entered = 1;
25         remoteDomain = _remoteDomain;
26         committedRoot = _committedRoot;
27         confirmAt[_committedRoot] = 1;
28    

In [ ]:
for d in os.listdir(): 
  if os.path.isdir(d) and d.startswith('lora-'):
      print(d)
      lora_model = d
  

##evaluating 8-bit quantized-insft llama7b trained with 50 epochs##

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
lora_param = '/content/drive/MyDrive/lora-llama7b_insft_line25e'
'''
for d in os.listdir(): 
     if os.path.isdir(d) and d.startswith('lora-'):
              if '7b' in d and 'insft' in d: 
                print(d)
                lora_param = d
'''

"\nfor d in os.listdir(): \n     if os.path.isdir(d) and d.startswith('lora-'):\n              if '7b' in d and 'insft' in d: \n                print(d)\n                lora_param = d\n"

In [19]:
import argparse
import random
import torch
#import gradio as gr
import torch.nn as nn
#import bitsandbytes
from datasets import Dataset
from peft import PeftConfig, LoraConfig, get_peft_model, PeftModel
from transformers import LlamaForCausalLM, LlamaTokenizer

def load_trained_llama(model_name):
  peft_model_id = model_name
  config = PeftConfig.from_pretrained(peft_model_id)
  model = LlamaForCausalLM.from_pretrained(config.base_model_name_or_path)
  model = PeftModel.from_pretrained(model, peft_model_id)
  tokenizer = LlamaTokenizer.from_pretrained(config.base_model_name_or_path)
  model.cuda()
  return model, tokenizer
def infer_invariants(model, tokenizer, test_data, \
                     temperature, top_p, top_k,  max_new_tokens):
  model.eval()
  inputs = tokenizer(test_data, return_tensors="pt")
  with torch.no_grad():
      outputs = model.generate(input_ids=inputs["input_ids"].cuda(), \
       temperature=temperature, 
       top_p=top_p, 
       top_k=top_k, 
      max_new_tokens=max_new_tokens)
      invariants = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
  return invariants

In [20]:
model, tokenizer = load_trained_llama(model_name=lora_param)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [25]:
contract = 'Replica.sol'
test_file = open(f'/content/drive/MyDrive/experiments/baseline benchmark/{contract}', "r")
test_data =  test_file.read() + \
            "\n" + "19+" + "\n" + "Given input contract and the program point, what's the invariant?"

In [26]:
output = infer_invariants(model=model, 
                          tokenizer=tokenizer, 
                          test_data=test_data,
                          temperature=None, 
                          top_p=None, 
                          top_k=None, 
                          max_new_tokens=512)
print(output.strip())
with open(f"/content/drive/MyDrive/experiments/evaluation/quantized_llama_inferred_invariants/{contract}", "w", encoding="utf-8") as file:  
    file.write(output.strip()+"\n")
file.close()

1 // SPDX-License-Identifier: MIT OR Apache-2.0
2 //test cross bridge
3 pragma solidity >=0.6.11;


4 contract Replica {
 
5     /**
6      * @notice Emitted when message is processed
7      * @param messageHash Hash of message that failed to process
8      * @param success TRUE if the call was executed successfully, FALSE if the call reverted
9      * @param returnData the return data from the external call
10      */
11     event Process(
12         bytes32 indexed messageHash,
13         bool indexed success,
14         bytes indexed returnData
15     );

   

16     function initialize(
17         uint32 _remoteDomain,
18         address _updater,
19         bytes32 _committedRoot,
20         uint256 _optimisticSeconds
21     ) public initializer {
22         __NomadBase_initialize(_updater);
23         // set storage variables
24         entered = 1;
25         remoteDomain = _remoteDomain;
26         committedRoot = _committedRoot;
27         confirmAt[_committedRoot] = 1;
28    

##Multi-Model CEL Evaluation##

In [ ]:
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import kaleido
import numpy as np
import pandas as pd
import simplejson as json
import plotly.express as px

In [ ]:
alpca_llama_data = '/content/drive/MyDrive/alpaca_llama_trained/trainer_state.json'
quantized_llama_data = '/content/drive/MyDrive/lora-llama7b_insft_50e/checkpoint-500/trainer_state.json'
opt_data = '/content/drive/MyDrive/opt350trainer.json'

In [ ]:
with open(f'{alpca_llama_data}') as json_file:
    alpaca_llama = json.load(json_file)
with open(f'{quantized_llama_data}') as json_file:
    quantized_llama = json.load(json_file)
with open(f'{opt_data}') as json_file:
    raw_opt = json.load(json_file)

In [ ]:
def delete_last_item(file): 
  new_data=[]
  count = (len(file) - 2)
  index = 0
  for item in file: 
    new_data.append(item)
    index += 1
    if index == count:
      break
  return new_data

def convert_2d_df(processed_df): 
    cols = ['epoch', 'cross entropy loss']
    epoch_df = []
    loss_df = []
    df = pd.DataFrame(columns=cols)
    for item in processed_df:
      epoch_df.append(item['epoch'])
      loss_df.append(item['loss'])
    df['cross entropy loss'] = loss_df
    df['epoch'] = epoch_df
    return df

def ready_to_plot(file1, file2, file3):
  data1 = delete_last_item(file1)
  data2 = delete_last_item(file2)
  data3 = delete_last_item(file3)
  ready_data1 = convert_2d_df(data1)
  ready_data2 = convert_2d_df(data2)
  ready_data3 = convert_2d_df(data3)
  return ready_data1,  ready_data2,  ready_data3

In [ ]:
alpaca, quantized, opt = ready_to_plot(alpaca_llama['log_history'], 
                          quantized_llama['log_history'],
                          raw_opt['log_history'])

In [ ]:
opt = opt[:31]
opt

,epoch,cross entropy loss
0,0.62,5.5122
1,1.62,14.9312
2,2.62,4.8242
3,3.62,5.7161
4,4.62,2.5249
5,5.62,2.6135
6,6.62,1.7229
7,7.62,1.7059
8,8.62,1.4524
9,9.62,2.1753


In [ ]:
new_row = pd.DataFrame({'epoch':'0.23', 'cross entropy loss':'2.15' }, index =[0])
quantized = pd.concat([new_row, quantized.loc[:]]).reset_index(drop=True) 


In [ ]:
t5 = pd.read_csv('/content/drive/MyDrive/t5-loss.csv')
gpt2 = pd.read_csv('/content/drive/MyDrive/gpt2-loss.csv')
t5= t5.drop(t5.columns[0], axis=1)
t5 = t5.rename({"epochs": "epoch", "cross entropy loss":"cross entropy loss"}, axis=1)
gpt2 = gpt2.drop(gpt2.columns[0], axis=1)
gpt2 = gpt2.rename({"epochs": "epoch", "cross entropy loss":"cross entropy loss"}, axis=1)
gpt2

,epoch,cross entropy loss
0,0,2.475849
1,1,2.489826
2,2,2.436219
3,3,2.440402
4,4,2.374811
5,5,2.330910
6,6,2.254919
7,7,2.217332
8,8,2.132691
9,9,2.065370


In [ ]:
!pip install --upgrade kaleido==0.2.0
!pip install psutil requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 3.7 MB/s eta 0:00:00
  Attempting uninstall: kaleido
    Found existing installation: kaleido 0.2.1
    Uninstalling kaleido-0.2.1:
      Successfully uninstalled kaleido-0.2.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import kaleido
# Create Line plot

fig = px.line(alpaca, x=alpaca['epoch'], y=alpaca['cross entropy loss'])
# Add Scatter plot
fig.data[0].name="Finetuned Alpaca_Llama"
fig.update_traces(showlegend=True)

fig.add_scatter(x=quantized['epoch'], 
                y=quantized['cross entropy loss'], 
                name ="Finetuned Quantized 8-bit Llama")

fig.add_scatter(x=opt['epoch'], 
                y=opt['cross entropy loss'],  
                name ="Finetuned Opt-350m")

fig.add_scatter(x=t5['epoch'], 
                y=t5['cross entropy loss'],  
                name ="Finetuned T5")

fig.add_scatter(x=gpt2['epoch'], 
                y=gpt2['cross entropy loss'],  
                name ="Finetuned GPT2")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.69
))
# Display the plot
fig.show()
#with open("/content/drive/MyDrive/model_loss.png", "w") as f:
#      f.write(fig.to_html())
#fig.write_image("model_loss.png", width=1920, height=1080)

##Seaborn Graphs Plotting##

In [1]:
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
